# **Final Project TN3156**
## **Authors: Matei Canavea, Casian Chiriac, Teodor Neagoe, Ivo van der Dussen (Group 13)**
# Two coupled TLS: the effect of decoherence on entanglement

## Introduction
One of the major difficulties of quantum computing is the decoherence of Two Level Systems (TLS). This loss of information is non reversible and thus creates issues with information processing inside a quantum computer. From this, it is easy to see why preventing decoherence is of great importance in this field of computing.<br><br>
One of the main causes of decoherence is noise. Noise causes dephasing in a qubit and thus leads to loss of information. For this project, a system of two coupled TLS's will be discussed. We will show how noise affects this two TLS system and simulate the system in QuTiP with varying parameters to find the most coherent system.<br><br>

## Description of the Physical Problem Being Simulated
Like mentioned before, our open system consists of two identical TLS's (e.g. Transmon Qubits, Spin Qubits etc.) which are coupled to eachother and thus, are able to exchange information. We will model the environment in 2 different ways: 1 collective "bath", which contains both TLS's and 2 seperate baths, each containing 1 TLS. In addition, we will also test 2 different kinds of subspaces: the singlet subspace and the triplet subspace. From testing all combinations of these parameters, we will be able to find the most noise-resistant configuration for our system.<br><br>

For this system, we use a Hamiltonian with some approximations: $H = H_0 + H_1 + H_{int}$. The important piece of this Hamiltonian is $H_{int}$. This interaction Hamiltonian looks like this after approximations: $H_{int} = J(\sigma_{+}^{(1)}\sigma_{-}^{(2)} + h.c.)$. For this Hamiltonian, the Rotating-Wave Approximation (RWA) was used. If we assume $J << \omega$ , we can neglect terms that don't conserve energy (like $\sigma_{+}^{(1)}\sigma_{+}^{(2)}$ and $\sigma_{-}^{(1)}\sigma_{-}^{(2)}$), since these terms oscillate rapidly, thus being able to be equated to 0. Also, we apply the TLS approximation, stating that the qubits are perfect Two Level Systems, thus ignoring leakage to higher levels (such as $\ket{2}$, $\ket{3}$ etc.). Finally, like mentioned before, we assume 2 identical qubits, with exactly the same energy levels.<br><br>

Based on physical intuition, we expect the system in the singlet subspace with the collective bath to be most resilient to noise, because of the anti-symmetrical nature of the singlet state and the symmetrical interaction to noise between both TLS's, respectively.<br><br>

## Aim and Scope of the Project
The main goal of this project is noise reduction. Using simulations inside of QuTiP, we will try to answer the following questions:
-  Is it possible to make the system consisting of two TLS's insensitive to noise with our parameters?
-  Against what kind of noise can our system be protected?
-  What would happen if we upscale from a bipartite system to a tripartite system?<br><br>

## Techniques and Approach
To answer these questions, we will plot concurrence graphs, to see how entanglement evolves through time while noise acts as decoherence. Concurrence is a measure of entanglement, ranging from 0 (no entanglement) to 1 (maximal entanglement). To calculate concurrence, we have defined a simplified interaction Hamiltonian using raising and lowering operators, like mentioned before. We use the concurrence() function, which is specifically defined for a 2-qubit system. The concurrence() function uses Wootters formula for calculating concurrence. This formula takes the density matrix of the system, multiplies it with it's own spin-flipped version and calculates all eigenvalues of the resulting matrix. All eigenvalues are then combined and a single value for concurrence is obtained: $C(\rho) = \max\{0, \lambda_1 - \lambda_2 - \lambda_3 ...\}$.<b><b>

To plot the concurrence against time, we use the Lindblad Master Equation to evolve the state vector $\Psi$ through time. The Lindblad Master Equation can be solved in QuTiP by using the function mesolve(). After supplying this function with arguments H (Hamiltonian), rho0 (initial state vector or density matrix) and c_ops (collapse operator, in our case $\sigma_-$), the function uses $\dot{\rho} = \frac{i}{\hbar}[H, \rho] + \sum\limits_{i}\gamma_{i}(L_{i} \rho L_{i}^{\dagger} - \frac{1}{2} \{L_{i}^{\dagger} L_{i}, \rho\})$.<b><b>

For additional visualisation, we also plot the Bloch sphere and the cityscape for each parameter:
-  The Bloch sphere can easily be plotted using bloch(). This class gives the user a blank canvas, which then can be filled with, for example, a density matrix. The class then, when requested, calculates the expectation values of the Pauli-matrices. It uses these values as 3D coordinates and plots vectors inside the Bloch sphere.
-  A cityscape of a density matrix is a 3D method of visualising the density matrix. Each element $\rho_{ij}$ in the matrix gets a "bulding". The height of the building is the absolute value of the real part of $\rho_{ij}$. All diagonal buildings correspond to the probability of finding the system in that specific state. All off-diagonal buildings correspond to the coherences. Due to this, the cityscape of a density matrix is an useful way of visualising evolution of coherence in the system.


For the "best" parameters, we will apply a gate while noise tries to decohere the system. This process will also be visualised using a Bloch sphere.<b><b>

## Results


In [ ]:
# -q flag used to suppress useless output
!pip -q install numpy matplotlib qutip ipywidgets

import numpy as np
from qutip import *
from ipywidgets import interact, FloatSlider
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from qutip import Bloch
from matplotlib import rc
rc('animation', html='jshtml')
plt.rcParams['animation.embed_limit'] = 90.0
from IPython.display import display, Image
from ipywidgets import interact, widgets
from IPython.display import display, clear_output

w = 1.0 * 2 * np.pi  # Qubit frequency = 1 Hz
J = 0.5 * 2 * np.pi  # Coupling strength between qubits = 0.5 Hz = the exchange (Vacuum Rabi) frequency
gamma = 0.2          # Decay rate (dissipation strength)
# Characteristic lifetime of the system: T1 ~ 1/gamma => "Death" happens after ~ 5s

times = np.linspace(0, 10, 200) # [0.0, 0.05, 0.1, ..., 9.95, 10.0]

# 2-qubit Hilber space (4 dimensions)
# |00> = |0> x |0> = (1,0,0,0)
# |01> = |0> x |1> = (0,1,0,0)
# |10> = |1> x |0> = (0,0,1,0)
# |11> = |1> x |1> = (0,0,0,1)

# Qubit 1 operators (act on 1st subspace and leave 2nd subspace unchanged)
sz1 = tensor(sigmaz(), qeye(2)) # Measure Z-Spin (here equivalent to measuring the Energy)
sp1 = tensor(sigmap(), qeye(2)) # Raising
sm1 = tensor(sigmam(), qeye(2)) # Lowering (decay)

# Qubit 2 operators (act on 2nd subspace and leave 1st subspace unchanged)
sz2 = tensor(qeye(2), sigmaz()) # Measure Z-Spin (here equivalent to measuring the Energy)
sp2 = tensor(qeye(2), sigmap()) # Raising
sm2 = tensor(qeye(2), sigmam()) # Lowering (decay)

# The Hamiltonian = Individual energies + Interaction term (Exchange interaction)
H = J * (sm1 * sp2 + sp1 * sm2)

## Scenario A: Independent Baths (The "Bad" Case)

When qubits are coupled to separate reservoirs (e.g., two distant atoms emitting spontaneously), decoherence attacks them individually.Mechanism: The environment effectively measures the state of each qubit independently, destroying the phase relationship between $|01\rangle$ and $|10\rangle$.Outcome: This is the primary scenario for ESD. The stronger the noise (amplitude damping or phase damping), the faster $C(t)$ hits zero.

## Scenario B: Common Bath (The "Interesting" Case)
When qubits effectively share an environment (e.g., two spins coupled to the same phonon mode), collective effects emerge. The phase of the noise acting on Qubit A is correlated with the noise on Qubit B.

### Superradiance (Fast Death)
If the qubits are in a symmetric state (like the triplet state $|\Psi_+\rangle = \frac{|01\rangle + |10\rangle}{\sqrt{2}}$), the noise interferes constructively.
- Effect: The system decays faster than a single independent qubit ($\Gamma_{collective} \approx 2\Gamma_{single}$). Entanglement dies very quickly.

### Subradiance & Decoherence-Free Subspaces (DFS)
If the qubits are in an antisymmetric state (like the singlet $|\Psi_-\rangle = \frac{|01\rangle - |10\rangle}{\sqrt{2}}$), the noise can interfere destructively.
- Effect: If the coupling is perfectly symmetric, the environment cannot distinguish between the two qubits to flip just one of them.
- Result: The state becomes "invisible" to the noise. The entanglement is protected and can theoretically persist indefinitely (or decay on much slower timescales), forming a Decoherence-Free Subspace (DFS).

In [ ]:
def get_bloch_vectors(result):
    # Define operators for Qubit 1 (acts on first space, identity on second)
    sx1 = tensor(sigmax(), qeye(2))
    sy1 = tensor(sigmay(), qeye(2))
    sz1 = tensor(sigmaz(), qeye(2))

    # Define operators for Qubit 2 (identity on first, acts on second
    sx2 = tensor(qeye(2), sigmax())
    sy2 = tensor(qeye(2), sigmay())
    sz2 = tensor(qeye(2), sigmaz())

    # Lists to store bloch vectors [x, y, z] for every time step
    vectors_q1 = []
    vectors_q2 = []

    for rho in result.states:
        # Calculate expectation values
        # Qubit 1
        v1 = [expect(sx1, rho), expect(sy1, rho), expect(sz1, rho)]

        # Qubit 2
        v2 = [expect(sx2, rho), expect(sy2, rho), expect(sz2, rho)]

        vectors_q1.append(v1)
        vectors_q2.append(v2)

    return np.array(vectors_q1), np.array(vectors_q2)

def visualize_electron_spins(result_A, sphere_color):
    # Get data
    vecs_q1, vecs_q2 = get_bloch_vectors(result_A)

    # Setup Figure
    fig = plt.figure(figsize=(6, 6))
    b = Bloch(fig=fig)
    b.vector_color = ['r', 'b']
    b.point_color = ['r', 'b']
    b.point_marker = ['o']
    b.sphere_color = sphere_color


    # Initial draw
    b.make_sphere()

    def animate(i):
        b.clear()

        # Qubit 1 (Red)
        b.add_vectors(vecs_q1[i])
        # only draw line if we have at least 2 points (i > 0)
        if i > 0:
            b.add_points([vecs_q1[:i+1, 0], vecs_q1[:i+1, 1], vecs_q1[:i+1, 2]], meth='l')

        # Qubit 2 (Blue)
        b.add_vectors(vecs_q2[i])
        if i > 0:
            b.add_points([vecs_q2[:i+1, 0], vecs_q2[:i+1, 1], vecs_q2[:i+1, 2]], meth='l')

        b.make_sphere()
        return []

    # Create Animation
    anim = animation.FuncAnimation(fig, animate, frames=len(vecs_q1), blit=False)
    print("Rendering GIF...")
    anim.save('spin_dynamics.gif', writer='pillow', fps=10)

    plt.close(fig)
    display(Image(filename='spin_dynamics.gif'))

def visualise_bloch_sphere(res):
    res_a, res_b = res
    visualize_electron_spins(res_a, '#0000FF15')
    visualize_electron_spins(res_b, '#FF800015')

def simulate(psi):
    # Baths. Used sp instead of sm, because sm made de states go to 1
    # Scenario A: Independent (Two separate "spies", blind to phase)
    c_ops_A = [np.sqrt(gamma) * sp1, np.sqrt(gamma) * sp2]

    # Scenario B: Common (One collective "spy", sensitive to phase)
    c_ops_B = [np.sqrt(gamma) * (sp1 + sp2)]

    # Simulations
    result_A = mesolve(H, psi, times, c_ops_A) # uses the Lindblad Master Equation
    result_B = mesolve(H, psi, times, c_ops_B)

    # Entanglement
    conc_A = [concurrence(rho) for rho in result_A.states]
    conc_B = [concurrence(rho) for rho in result_B.states]

    plt.figure(figsize=(10, 6))
    plt.plot(times, conc_A, label='A: Independent (Two separate "spies", blind to phase)')
    plt.plot(times, conc_B, label='B: Common (One collective "spy", sensitive to phase)')
    plt.axhline(0, color='k', linewidth=0.5)
    plt.title(f'Decoherence Effect on: {psi.full()}')
    plt.xlabel('Time (s)')
    plt.ylabel('Entanglement (Concurrence)')
    plt.legend()
    plt.grid(True)
    plt.show()

    return result_A, result_B

# Initial States
# Tried to use Qobj([0, 1, 1, 0]), but then you need to manually specify dims
# Symmetric (Triplet) - "Bright"
psi_bright = (ket("01") + ket("10")).unit()

# Antisymmetric (Singlet) - "Dark"
psi_dark = (ket("01") - ket("10")).unit()

# visualise_bloch_sphere(simulate(psi_bright))

# visualise_bloch_sphere(simulate(psi_dark))



In [ ]:
res = None

def run_simplified_control(amp00, amp01, amp10, amp11):
    global res
    psi_raw = (amp00 * ket("00") +
               amp01 * ket("01") +
               amp10 * ket("10") +
               amp11 * ket("11"))

    # .unit() gives ZeroDivisionError when norm is 0
    if psi_raw.norm() == 0:
        psi0 = ket("00")
    else:
        psi0 = psi_raw.unit()

    res = simulate(psi0)

interact(run_simplified_control,
         amp00=FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description='|00>'),
         amp01=FloatSlider(value=1.0, min=-1.0, max=1.0, step=0.1, description='|01>'),
         amp10=FloatSlider(value=1.0, min=-1.0, max=1.0, step=0.1, description='|10>'),
         amp11=FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.1, description='|11>')
)

def visualise_cityscape(res):
    """Create cityscape animations for both scenarios A and B"""
    from mpl_toolkits.mplot3d import Axes3D
    
    res_a, res_b = res
    
    labels = ['|00⟩', '|01⟩', '|10⟩', '|11⟩']
    n = 4
    xpos, ypos = np.meshgrid(np.arange(n), np.arange(n))
    xpos = xpos.flatten()
    ypos = ypos.flatten()
    zpos = np.zeros(n * n)
    dx = dy = 0.6 * np.ones(n * n)
    
    # Color: diagonal = blue (populations), off-diagonal = red (coherences)
    colors_base = []
    for i in range(n):
        for j in range(n):
            if i == j:
                colors_base.append('#2563eb')  # Blue for diagonal (populations)
            else:
                colors_base.append('#dc2626')  # Red for off-diagonal (coherences)
    
    def create_cityscape_gif(result, title, filename):
        fig = plt.figure(figsize=(8, 6))
        
        def update(frame):
            fig.clf()
            ax = fig.add_subplot(111, projection='3d')
            
            rho = result.states[frame]
            rho_abs = np.abs(rho.full()).flatten()
            
            # Vary alpha based on magnitude for visual effect
            colors = []
            for idx, (c, val) in enumerate(zip(colors_base, rho_abs)):
                alpha = max(0.3, min(1.0, val + 0.3))
                if colors_base[idx] == '#2563eb':
                    colors.append((0.145, 0.388, 0.922, alpha))  # Blue with alpha
                else:
                    colors.append((0.863, 0.149, 0.149, alpha))  # Red with alpha
            
            ax.bar3d(xpos, ypos, zpos, dx, dy, rho_abs, color=colors, edgecolor='black', linewidth=0.3)
            
            ax.set_xlabel('Column')
            ax.set_ylabel('Row')
            ax.set_zlabel('|ρᵢⱼ|')
            ax.set_zlim(0, 0.6)
            ax.set_xticks(np.arange(n) + 0.3)
            ax.set_yticks(np.arange(n) + 0.3)
            ax.set_xticklabels(labels)
            ax.set_yticklabels(labels)
            ax.view_init(elev=25, azim=45)
            ax.set_title(f"{title}\nt = {times[frame]:.2f}s\n[BLUE] Populations    [RED] Coherences", fontsize=10)
        
        ani = animation.FuncAnimation(fig, update, frames=len(result.states), interval=80)
        print(f"Rendering {filename}...")
        ani.save(filename, writer='pillow', fps=12)
        plt.close(fig)
        return filename
    
    # Create both GIFs
    gif_a = create_cityscape_gif(res_a, 'Scenario A: Independent Baths', 'cityscape_independent.gif')
    gif_b = create_cityscape_gif(res_b, 'Scenario B: Common Bath', 'cityscape_common.gif')
    
    # Display both
    print("\n🏙️ Cityscape A: Independent Baths (Two separate 'spies')")
    display(Image(filename=gif_a))
    print("\n🏙️ Cityscape B: Common Bath (One collective 'spy')")
    display(Image(filename=gif_b))

btn_viz = widgets.Button(
    description='Visualize Bloch Spheres (3D)',
    button_style='warning',
    icon='video-camera',
    layout=widgets.Layout(width='50%')
)
out_viz = widgets.Output()
def on_btn_click(b):
    global res
    with out_viz:
        clear_output()
        visualise_bloch_sphere(res)

btn_viz.on_click(on_btn_click)

btn_cityscape = widgets.Button(
    description='Visualize Cityscape (3D Density Matrix)',
    button_style='info',
    icon='bar-chart',
    layout=widgets.Layout(width='50%')
)
out_cityscape = widgets.Output()
def on_cityscape_click(b):
    global res
    with out_cityscape:
        clear_output()
        visualise_cityscape(res)

btn_cityscape.on_click(on_cityscape_click)

display(widgets.VBox([
    btn_viz,
    out_viz,
    btn_cityscape,
    out_cityscape
]))

# interesting states configs:
# |00> 1, |01> 1, |10> -1, |11> -1      # opposing vectors converging downwards
# |00> 0, |01> 1, |10> 0, |11> 0        # vectors started opposing, converge on half of the length to |1>

## Visualizing Decoherence-Free Subspaces (DFS)

The **singlet state** $|\Psi_-\rangle = \frac{|01\rangle - |10\rangle}{\sqrt{2}}$ lives in a decoherence-free subspace when coupled to a common bath. By rotating continuously from the singlet to the triplet state $|\Psi_+\rangle = \frac{|01\rangle + |10\rangle}{\sqrt{2}}$, we can observe how **leaving the DFS introduces decoherence**.

The rotation is parameterized by angle $\theta$:
$$|\Psi(\theta)\rangle = \cos(\theta)|\Psi_-\rangle + \sin(\theta)|\Psi_+\rangle$$

- At $\theta = 0$: Pure singlet (DFS protected)
- At $\theta = \pi/2$: Pure triplet (maximally exposed to decoherence)
- Intermediate angles: Partial protection


In [ ]:
from IPython.display import HTML
import os

# Create output folder for GIFs
GIF_FOLDER = "basis_change"
os.makedirs(GIF_FOLDER, exist_ok=True)

def rotate_basis(theta):
    """Rotate from singlet (theta=0, DFS protected) to triplet (theta=π/2, not protected)"""
    psi_singlet = (ket("01") - ket("10")).unit()
    psi_triplet = (ket("01") + ket("10")).unit()
    psi = np.cos(theta) * psi_singlet + np.sin(theta) * psi_triplet
    return psi.unit()

def simulate_dfs(theta):
    """Simulate decoherence with common bath"""
    psi = rotate_basis(theta)
    c_ops = [np.sqrt(gamma) * (sm1 + sm2)]
    result = mesolve(H, psi, times, c_ops)
    concurrence_vals = [concurrence(rho) for rho in result.states]
    return result, concurrence_vals

def density_animation(result, theta_deg):
    """Animate density matrix evolution (2D heatmap) and save as GIF"""
    fig, ax = plt.subplots(figsize=(4, 4))

    def update(i):
        ax.clear()
        rho = result.states[i]
        ax.imshow(np.abs(rho.full()), cmap='viridis', vmin=0, vmax=1)
        ax.set_title(f"Density Matrix  |  t = {times[i]:.2f}")
        ax.set_xticks([0,1,2,3])
        ax.set_yticks([0,1,2,3])

    ani = animation.FuncAnimation(fig, update, frames=len(times), interval=80)
    
    # Save GIF
    gif_path = os.path.join(GIF_FOLDER, f"theta_{theta_deg:.0f}_density.gif")
    print(f"   Saving density animation to {gif_path}...")
    ani.save(gif_path, writer='pillow', fps=12)
    plt.close(fig)
    return gif_path

def cityscape_animation(result, theta_deg):
    """Animate density matrix as 3D bar chart and save as GIF"""
    from mpl_toolkits.mplot3d import Axes3D
    
    labels = ['|00⟩', '|01⟩', '|10⟩', '|11⟩']
    n = 4
    xpos, ypos = np.meshgrid(np.arange(n), np.arange(n))
    xpos = xpos.flatten()
    ypos = ypos.flatten()
    zpos = np.zeros(n * n)
    dx = dy = 0.6 * np.ones(n * n)
    
    # Color: diagonal = blue (populations), off-diagonal = red/orange (coherences)
    colors_base = []
    for i in range(n):
        for j in range(n):
            if i == j:
                colors_base.append('#2563eb')  # Blue for diagonal (populations)
            else:
                colors_base.append('#dc2626')  # Red for off-diagonal (coherences)
    
    fig = plt.figure(figsize=(8, 6))
    
    def update(frame):
        fig.clf()
        ax = fig.add_subplot(111, projection='3d')
        
        rho = result.states[frame]
        rho_abs = np.abs(rho.full()).flatten()
        
        # Vary alpha based on magnitude for visual effect
        colors = []
        for idx, (c, val) in enumerate(zip(colors_base, rho_abs)):
            alpha = max(0.3, min(1.0, val + 0.3))
            if colors_base[idx] == '#2563eb':
                colors.append((0.145, 0.388, 0.922, alpha))  # Blue with alpha
            else:
                colors.append((0.863, 0.149, 0.149, alpha))  # Red with alpha
        
        ax.bar3d(xpos, ypos, zpos, dx, dy, rho_abs, color=colors, edgecolor='black', linewidth=0.3)
        
        ax.set_xlabel('Column')
        ax.set_ylabel('Row')
        ax.set_zlabel('|ρᵢⱼ|')
        ax.set_zlim(0, 0.6)
        ax.set_xticks(np.arange(n) + 0.3)
        ax.set_yticks(np.arange(n) + 0.3)
        ax.set_xticklabels(labels)
        ax.set_yticklabels(labels)
        ax.view_init(elev=25, azim=45)
        ax.set_title(f"Density Matrix Cityscape  |  t = {times[frame]:.2f}s\n"
                     f"[BLUE] Populations    [RED] Coherences", fontsize=10)
    
    ani = animation.FuncAnimation(fig, update, frames=len(result.states), interval=80)
    
    # Save GIF
    gif_path = os.path.join(GIF_FOLDER, f"theta_{theta_deg:.0f}_cityscape.gif")
    print(f"   Saving cityscape animation to {gif_path}...")
    ani.save(gif_path, writer='pillow', fps=12)
    plt.close(fig)
    return gif_path

def bloch_animation(result, theta_deg):
    """Animate Bloch sphere evolution for both qubits and save as GIF"""
    reduced1 = [rho.ptrace(0) for rho in result.states]
    reduced2 = [rho.ptrace(1) for rho in result.states]
    fig = plt.figure(figsize=(8, 4))

    def update(i):
        fig.clf()
        ax1 = fig.add_subplot(121, projection='3d')
        ax2 = fig.add_subplot(122, projection='3d')

        b1 = Bloch(axes=ax1)
        b2 = Bloch(axes=ax2)
        b1.add_states(reduced1[i])
        b2.add_states(reduced2[i])
        b1.render()
        b2.render()

        ax1.set_title("Qubit 1")
        ax2.set_title("Qubit 2")

    ani = animation.FuncAnimation(fig, update, frames=len(times), interval=80)
    
    # Save GIF
    gif_path = os.path.join(GIF_FOLDER, f"theta_{theta_deg:.0f}_bloch.gif")
    print(f"   Saving Bloch sphere animation to {gif_path}...")
    ani.save(gif_path, writer='pillow', fps=12)
    plt.close(fig)
    return gif_path

def dfs_dashboard(theta_deg):
    """Main interactive dashboard for DFS visualization"""
    theta = np.radians(theta_deg)

    result, conc = simulate_dfs(theta)
    _, conc_singlet = simulate_dfs(0)
    _, conc_triplet = simulate_dfs(np.pi/2)

    fig, axes = plt.subplots(1, 3, figsize=(17, 4))

    # Panel 1: State composition
    psi = rotate_basis(theta)
    probs = np.abs(psi.full().flatten())**2
    axes[0].bar(['|00⟩','|01⟩','|10⟩','|11⟩'], probs,
                color=['#1a1a2e','#16213e','#0f3460','#e94560'])
    axes[0].set_ylim(0, 0.6)
    axes[0].set_title(f"State Composition (θ={theta_deg:.0f}°)", fontweight='bold')
    axes[0].set_ylabel('|Amplitude|²')

    # Panel 2: DFS Protection Gauge
    protection = np.cos(theta)**2
    x = np.linspace(0, np.pi, 200)
    axes[1].plot(np.cos(x), np.sin(x), 'k-', lw=3)
    needle_angle = np.pi * (1 - protection)
    axes[1].arrow(0, 0, 0.8 * np.cos(needle_angle), 0.8 * np.sin(needle_angle),
                  head_width=0.1, head_length=0.05, fc='red', ec='red', lw=3)
    axes[1].set_aspect('equal')
    axes[1].set_xlim(-1.2, 1.2)
    axes[1].set_ylim(-0.2, 1.2)
    axes[1].axis('off')
    axes[1].set_title(f"DFS Protection: {protection:.0%}", fontweight='bold')

    # Panel 3: Concurrence decay
    axes[2].fill_between(times, conc_singlet, conc_triplet, color='gray', alpha=0.2)
    axes[2].plot(times, conc_singlet, 'g--', label='DFS')
    axes[2].plot(times, conc_triplet, 'r:', label='Triplet')
    axes[2].plot(times, conc, 'b-', lw=3, label=f"θ={theta_deg}°")
    axes[2].set_title("Entanglement Decay (Common Bath)", fontweight='bold')
    axes[2].set_xlabel("Time")
    axes[2].set_ylabel("Concurrence")
    axes[2].set_ylim(-0.05, 1.05)
    axes[2].grid(True, alpha=0.2)
    axes[2].legend()

    plt.tight_layout()
    plt.show()

    print("\n⏳ Generating animations (this may take a moment)...")
    print("=" * 50)
    
    print("\n🏙️ Density Matrix Cityscape (3D Bar Chart):")
    print("   Watch the red coherence bars crumble while blue population bars shift!")
    cityscape_path = cityscape_animation(result, theta_deg)
    display(Image(filename=cityscape_path))

    print("\n💠 Density Matrix Heatmap:")
    density_path = density_animation(result, theta_deg)
    display(Image(filename=density_path))

    print("\n💠 Bloch Sphere Animation:")
    bloch_path = bloch_animation(result, theta_deg)
    display(Image(filename=bloch_path))
    
    print("\n" + "=" * 50)
    print(f"✅ All animations saved to '{GIF_FOLDER}/' folder!")
    print(f"   - {os.path.basename(cityscape_path)}")
    print(f"   - {os.path.basename(density_path)}")
    print(f"   - {os.path.basename(bloch_path)}")

interact(
    dfs_dashboard,
    theta_deg=FloatSlider(
        value=0, min=0, max=90, step=5,
        description="θ (degrees): DFS → Non-DFS",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )
);


## Gallery of Interesting Quantum States

Below we showcase several interesting initial states and their complete dynamics:
1. **|01⟩ + |10⟩**: Symmetric triplet state (bright state) - base case
2. **|01⟩ - |10⟩**: Antisymmetric singlet state (dark state) - DFS protected
3. **|01⟩**: Single basis state - simple separable state
4. **|00⟩ + |01⟩ + |10⟩ + |11⟩**: Uniform superposition of all basis states

For each state, we show:
- **Concurrence graph**: How entanglement decays over time
- **Bloch sphere animations**: Spin dynamics for both qubits
- **Cityscape animations**: 3D density matrix evolution (populations in blue, coherences in red)


In [ ]:
def showcase_state(state_name, psi, case_number):
    """Display complete visualization suite for a given quantum state"""
    print("=" * 80)
    print(f"\nCase {case_number}: {state_name}")
    print("=" * 80)
    
    # Run simulation and show concurrence graph
    print("\n📊 CONCURRENCE EVOLUTION:")
    res = simulate(psi)
    
    # Show Bloch sphere animations
    print("\n🌐 BLOCH SPHERE DYNAMICS:")
    visualise_bloch_sphere(res)
    
    # Show cityscape animations
    print("\n🏙️ DENSITY MATRIX CITYSCAPE:")
    visualise_cityscape(res)
    
    print("\n" + "=" * 80)
    print(f"✅ Case {case_number} complete!\n")

# Define the interesting states
print("🚀 Starting gallery of interesting quantum states...\n")

# Case 1: |01⟩ + |10⟩ (Triplet/Bright State - Base Case)
psi_1 = (ket("01") + ket("10")).unit()
showcase_state("|01⟩ + |10⟩ (Symmetric Triplet - Bright State)", psi_1, 1)

# Case 2: |01⟩ - |10⟩ (Singlet/Dark State - DFS Protected)
psi_2 = (ket("01") - ket("10")).unit()
showcase_state("|01⟩ - |10⟩ (Antisymmetric Singlet - DFS Protected)", psi_2, 2)

# Case 3: |01⟩ (Single Basis State)
psi_3 = ket("01")
showcase_state("|01⟩ (Single Basis State - Separable)", psi_3, 3)

# Case 4: |00⟩ + |01⟩ + |10⟩ + |11⟩ (Uniform Superposition)
# psi_4 = (ket("00") + ket("01") + ket("10") + ket("11")).unit()
# showcase_state("|00⟩ + |01⟩ + |10⟩ + |11⟩ (Uniform Superposition of All States)", psi_4, 4)

print("\n" + "🎉" * 40)
print("Gallery complete! All interesting cases have been visualized.")
print("🎉" * 40)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from ipywidgets import interact, FloatSlider, Checkbox, Dropdown
import ipywidgets as widgets

# ==============================================================================
# 1. SUPPORTING FUNCTIONS (Projection & Visualisation)
# ==============================================================================

def project_to_logical_subspace(states):
    """
    Projects 4x4 matrix to 2x2 logical matrix
    Logical 0 = |01>, Logical 1 = |10>
    """
    logical_states = []
    for rho in states:
        if isket(rho): rho = ket2dm(rho)
        data = rho.full()
        
        # Project on index 1 (|01>) and index 2 (|10>)
        rho_logical_data = np.array([
            [data[1,1], data[1,2]],
            [data[2,1], data[2,2]]
        ])
             
        logical_states.append(Qobj(rho_logical_data))
    return logical_states

def visualize_logical_bloch(result, title, color_style):
    logical_states = project_to_logical_subspace(result.states)
    
    xp, yp, zp = [], [], []
    for rho in logical_states:
        xp.append(float(np.real(expect(sigmax(), rho))))
        yp.append(float(np.real(expect(sigmay(), rho))))
        zp.append(float(np.real(expect(sigmaz(), rho))))
    
    fig = plt.figure(figsize=(6, 6))
    b = Bloch(fig=fig)
    b.zlabel = ['$|0_L\\rangle$', '$|1_L\\rangle$']
    b.xlabel = ['$|+\\rangle_L$', '$|-\\rangle_L$']
    b.title = title
    b.point_color = [color_style]; b.point_marker = ['o']; b.point_size = [20]
    b.vector_color = [color_style]; b.view = [-40, 30]
    
    if len(xp) > 0:
        b.add_points([xp, yp, zp], meth='s')
        b.add_vectors([xp[-1], yp[-1], zp[-1]])
    
    b.show()
    plt.close() # Prevents double plot

# ==============================================================================
# 2. MAIN FUNCTION (SIMULATION)
# ==============================================================================

def run_advanced_demo(gamma_val, enable_phase, enable_energy, protection_mode):
    
    # --- A. Setup ---
    psi_0L = tensor(basis(2, 0), basis(2, 1)) # Start op |01> (Noordpool)
    times = np.linspace(0, 10, 100)
    
    # Hamiltonian (Logical X-gate / Exchange Interaction)
    w = 1.0 * 2 * np.pi
    J = 0.5 * 2 * np.pi
    
    # Define operators
    sz1 = tensor(sigmaz(), qeye(2)); sz2 = tensor(qeye(2), sigmaz())
    sm1 = tensor(sigmam(), qeye(2)); sm2 = tensor(qeye(2), sigmam())
    sp1 = tensor(sigmap(), qeye(2)); sp2 = tensor(qeye(2), sigmap())
    
    H_sys = 0.5 * w * (sz1 + sz2) + J * (sm1 * sp2 + sp1 * sm2)
    
    # --- B. Noise ---
    c_ops = []
    is_common = (protection_mode == 'Common Bath (DFS)')
    
    # 1. Phase Noise (Dephasing - Z)
    if enable_phase:
        if is_common:
            # DFS beschermt hiertegen! (Z1 + Z2) op |01> is 0.
            c_ops.append(np.sqrt(gamma_val) * (sz1 + sz2))
        else:
            # Separate bath: funest voor fase
            c_ops.append(np.sqrt(gamma_val) * sz1)
            c_ops.append(np.sqrt(gamma_val) * sz2)
            
    # 2. Energy Loss (Dissipation - Sigma Minus)
    if enable_energy:
        if is_common:
            # DFS beschermt hier NIET tegen! (Lekt naar |00>)
            c_ops.append(np.sqrt(gamma_val) * (sm1 + sm2))
        else:
            # Separate bath: ook lekkage
            c_ops.append(np.sqrt(gamma_val) * sm1)
            c_ops.append(np.sqrt(gamma_val) * sm2)

    # Title and color plot
    if is_common:
        title = "Mode: DFS Protected"
        color = 'g'
    else:
        title = "Mode: Unprotected"
        color = 'r'
        
    if not c_ops:
        title += " (No Noise)"

    # --- C. Simulation ---
    result = mesolve(H_sys, psi_0L, times, c_ops, [])
    
    # --- D. Visualisation Bloch Sphere ---
    visualize_logical_bloch(result, title, color)
    
    # --- E. Purity Plot (Vector Length) ---
    log_states = project_to_logical_subspace(result.states)
    purities = []
    
    for rho in log_states:
        vx = float(expect(sigmax(), rho))
        vy = float(expect(sigmay(), rho))
        vz = float(expect(sigmaz(), rho))
        
        length = np.sqrt(vx**2 + vy**2 + vz**2)
        purities.append(length)
    
    plt.figure(figsize=(10,4))
    plt.plot(times, purities, color=color, linewidth=2, label='Measured Purity')
    plt.ylim(0, 1.1)
    plt.title("Logical Vector Length (Information Retention)")
    plt.xlabel("Time")
    plt.ylabel("Length (r)")
    plt.axhline(1.0, color='k', linestyle='--', alpha=0.5, label='Ideal Unitary')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# ==============================================================================
# 3. INTERACTIVE WIDGET
# ==============================================================================
interact(run_advanced_demo, 
         gamma_val=FloatSlider(min=0.0, max=1.0, step=0.1, value=0.3, description="Noise Strength"),
         enable_phase=Checkbox(value=True, description="Add Phase Noise (Z)"),
         enable_energy=Checkbox(value=False, description="Add Energy Loss (Sm)"),
         protection_mode=Dropdown(
             options=['Common Bath (DFS)', 'Separate Bath (Unprotected)'],
             value='Common Bath (DFS)',
             description='Mode:',
         )
);